In [ ]:
import os

if not os.path.exists('engine.py'):
  !wget https://raw.githubusercontent.com/jpneto/yocto/main/src/codepage.py
  !wget https://raw.githubusercontent.com/jpneto/yocto/main/src/operators.py
  !wget https://raw.githubusercontent.com/jpneto/yocto/main/src/engine.py
  !> __init__.py



---



## Tacit Programming

In tacit programming defining a function is not done by making its arguments explicit but, as we will see, by applying function composition.

When defining a regular function, in Python or in most languages, we need to be explicit with its arguments,

In [ ]:
def binarySum(x,y): # x and y are the arguments of binarySum
  return x+y

In functional languages like Haskell, it is possible to create partial functions, i.e., functions that only need a subset of the original function arguments. This process can be used to create versions of the original functions without arguments. Check this [example](https://en.wikipedia.org/wiki/Tacit_programming#Functional_programming).

Python does not natively support partial functions, but we can do a similar process with function `partial`,

In [ ]:
from functools import partial, reduce

# partial can only replace the leftmost arguments
successor = partial(binarySum, 1) # same as binarySum(1,y)
successor(3)

A related technique is _function currying_, the technique of converting a function accepting a tuple of arguments into a sequence of unary functions,

In [ ]:
def ternarySum(x,y,z):
  return x+y+z

ternarySumCurried = lambda x : lambda y : lambda z : x+y+z
ternarySumCurried(4)(6)(5)

A partial function is the evaluation of a curried function at a fixed argument. So, any time we fix an argument on a curried function, we get a partial function,

In [ ]:
successor_v2 = ternarySumCurried(0)(1)
successor_v2(3)

Despite the limitations of Python, it is still possible to achieve some degree of tacit programming using lambda expressions and some high-order functions.

Let's say I wish to define the following function in a tacit way:

In [ ]:
from math import *

def fun(xs):
  y = sum(xs)
  z = sqrt(y)
  w = abs(z)
  return w

fun([1,2,3])  

How to redefine `fun`?

We can try to surround the function calls inside each other,

In [ ]:
fun1 = lambda xs : abs( sqrt ( sum ( xs ) ) )
fun1([1,2,3])

But the parameters are still there and this is not practical if the number of functions is more than a few.

We can define a composition function and use it to join pairs of functions,

In [ ]:
# composition of two functions
composition = lambda f,g : lambda x: f(g(x))

fun2 = composition(abs, composition(sqrt, sum))
fun2([1,2,3])

But the pratical problem of having too many internal calls remains.

However, there is a pattern here. To compose $n$ functions `[f_1,f_2,…,f_n]` the expression becomes

`composition(f1, composition(f2, … composition(fn-1,fn)…))`

We can apply a fold operation to chain all these compositions. In Python the high-order function that computes this pattern is denoted `reduce`,

In [ ]:
def compose(fs):
  return reduce(composition, fs)

fun3 = compose([abs, sqrt, sum])  
fun3([1,2,3])

The last gotcha is that the function list order is the reverse of our intuition (the first function is `sum` which comes in last), so let's create an helper function that reverses the function list,

In [ ]:
def stream(fs):
  return compose(fs[::-1])

fun4 = stream([sum, sqrt, abs])  
fun4([1,2,3])     # can be read as [1,2,3] >> sum >> sqrt >> abs >> final result

We were able to stream-line the composition of several functions in a natural way. No more intermediate arguments are needed.

One well-known example of tacit programming is the Unix/Linux pipe system that invokes programs in sequence, feeding the output of one program into the input of the next, This can be seen as a model for data flow processing.

In [ ]:
!cat operators.py | sort | uniq -c | sort -rn | head -n8

The pipe `|` works as the composition operator. The processed data are never mentioned.

## A Python class for combinatorial operators

We are going to present a Python class that uses operator overloading and the capabilities shown above to implement a syntax closer to tacit languages, namely by implementing [combinatorial operators](https://en.wikipedia.org/wiki/SKI_combinator_calculus), in a similar fashion as in APL, Haskell and BQN.

This code was initially based on a solution found at [Samuel's github repository](https://gist.github.com/Demonstrandum).

In [1]:
from inspect import signature
from functools import partial
from collections.abc import Iterable

Each combinator is going to be an operator over objects of class `Tacit`. These objects are functions that are transformed by appliying these operators.

In [2]:
class Tacit(object):
  def __init__(self, λ, arity=None):
      self.λ = λ
      self.arity = len(signature(λ).parameters) if arity is None else arity

  def __mul__(self, λ2):
    """ * is the B combinator (composition); Bxyz ≡ x(yz) """
    return Tacit( lambda *xs : self.λ(λ2(*xs)) )
  
  def __pow__(self, λ2):
    """ ** is the Over (APL funcional); f**g(x,y) ≡ f(g(x),g(y)) """
    return Tacit( lambda x,y : self.λ(λ2(x), λ2(y)) )
    
  def __or__(self, λ2):      
    """ | is the S combinator; Sxyz ≡ (xz)(yz) """
    return Tacit( lambda z : self.λ(z, λ2(z)) )
    
  def __and__(self, λ2):      
    """ & is the K combinator; Kxy ≡ x """
    return Tacit( self.λ )
    
  def __invert__(self):
    """ ~ is the C operator (flip args); Cxyz ≡ xzy """
    return Tacit( lambda *xs : self.λ(*(xs[::-1])), self.arity )
  
  def __gt__(self, λ2):
    """ > is the Before (BQN funcional); g>f(x,y) ≡ f(g(x),y) """
    return Tacit( lambda x,y : λ2(self.λ(x), y) )
  
  def __lt__(self, λ2):
    """ < is the After (BQN funcional); f<g(x,y) ≡ f(x,g(y)) """
    return Tacit( lambda x,y : self.λ(x, λ2(y)) )
  
  def __pos__(self):
    """ Y combinator """
    f = lambda x: self.λ(lambda z: x(x)(z))
    return Tacit( f(f) )
  
  def __rshift__(self, λ2):
    """ a kind-of pipe, needs to lift values before function application """
    value = self.λ()
    if isinstance(value, Iterable):
      return Tacit( lambda : λ2(*value) )
    else:
      return Tacit( lambda : λ2(value) )
  
  def __call__(self, *arg):
    # if not all args are present, returns a curried tacit function
    if len(arg) < self.arity:
      return Tacit( partial(self.λ, *arg), self.arity-len(arg) )
    return self.λ(*arg)

  def make(f):
    """ decorator """
    return Tacit(f)

Besides these operations, let's include a lift function that lifts type literals (like ints and tuples) to become tacit (constant) functions:

In [8]:
@Tacit.make
def l(*literal):
  """ lifts a literal to a tacit constant function """
  if len(literal) == 1:
    return Tacit( lambda : literal[0] )
  return Tacit( lambda : literal )

Now we can define some functions and check how they can be combined:

In [4]:
@Tacit.make
def f(x, y):
  return 2 * x + y

@Tacit.make
def g(x): 
  return x + 3

To compose functions we 'multiply' them:

In [6]:
h1 = g*g
assert h1(1) == 7  # h1(1) ≡ g(g(1))

Another examples:

In [9]:
h2  = f|g          # h2(1)    ≡ f(1, g(1))
h3  = ~f           # h3(1,2)  ≡ f(2, 1)
h4  = f**g         # h4(1,2)  ≡ f(g(1), g(2))
h5  = g>f          # h5(1,2)  ≡ f(g(1), 2)
h5a = g>~f         # h5a(1,2) ≡ f(2, g(1))
h6  = f<g          # h6(1,2)  ≡ f(1, g(2)); same as f|g
h7  = l(5)>>g>>g   # h7()     ≡ g(g(5))
h8  = l(5,1)>>f>>g # h8()     ≡ g(f(5, 1))

To test the Y combinator (our _tour de force_), we use the classical factorial:

In [10]:
@Tacit.make
def F(f):
  """ the fix point of this 'loop' is the recursive factorial function """
  return lambda n: 1 if n==0 else n*f(n-1)

h9 = +F # h9 becomes the recursive factorial (!)
assert h9(5) == 120

Other [birds are possible](https://www.angelfire.com/tx4/cus/combinator/birds.html), but this is enough for a proof of concept.



---



## Concatenative Programming

Consider you need to check if a given register satisfies lots of different predicates, and don't want to maintain a maze of if-then-elses.

Using what we learn , we can try to streamline the required validations by passing the register across those predicates, together with a boolean that accumulates the successive validations,

In [ ]:
# each predicate receive a pair (register, isRegisterStillValid?)
correctName  = lambda pair : ( pair[0], pair[1] and len(pair[0]['name']) > 4 )
enoughScore  = lambda pair : ( pair[0], pair[1] and pair[0]['score'] >= 50   )
isLegal      = lambda pair : ( pair[0], pair[1] and pair[0]['legal']         )

validations = [correctName, enoughScore, isLegal]

validate = lambda register : stream(validations)( (register,True) )[1]

aRegister = { 'name':'João Neto', 'score':100, 'legal':True }
validate(aRegister)

There are simpler solutions in Python, like evaluating predicate `all` after mapping a list of predicates on the given register. 

But if we needed, for some reason, to change the register during the processing flow, our solution could be easily adapted.

Now let's take this example and modify it so that we replace predicates with general functions, and replace tuples/registers with a data structure able to keep the cumulative results of executing these general functions.

What's to be gain here? Well, nothing more nothing less that a prototype for a programming language!

A typical data structure used in this context is the stack. We will use a list to represent a stack, restricting ourselves by using only `append` and `pop` to access the end of the list.

Here is an example using constants, binary addition and multiplication to compute arithmetic expressions via the stack

In [ ]:
# insertN returns a function that, when called, inserts n into the stack/state
def insertN(n):
  def push(state):
    state.append(n)
    return state
  return push

def add(state):
  b = state.pop()
  a = state.pop()
  state.append(a+b)
  return state

def mul(state):
  b = state.pop()
  a = state.pop()
  state.append(a*b)
  return state

execute = lambda program, state : stream(program)(state)

initialState = [] # initially we have an empty state
program = [insertN(3), insertN(7), mul, insertN(10), add]  # computes 3*7+10

execute(program, initialState)

This is a way of programming that, by its nature, is tacit programming. The numbers we used in the previous example are not arguments, they represent functions that insert constant values into the state. Everything is a function, and every program is a composition of functions.

Languages like this are part of the programming paradigm called **concatenative programming**.

Check [concatenative.org](https://concatenative.org/wiki/view/Concatenative%20language) for more information. Notable examples include [Forth](https://concatenative.org/wiki/view/Forth) (the original concatenative language), [Joy](https://www.latrobe.edu.au/humanities/research/research-projects/past-projects/joy-programming-language), [PostScript](https://en.wikipedia.org/wiki/PostScript), and [Factor](https://factorcode.org/).



---



The following examples are based on a code-golfing concatenative language written in Python (so I can run it in this notebook) called Yöctọ.

In [ ]:
from engine import runProgram
from fractions import Fraction

def run(program, inputs=[]):
  commands = [ line for line in program.split(sep='\n') if line != '' ]
  inputs = [Fraction(i) for i in inputs]
  return runProgram(commands, inputs)

Like many concatenative languages, Yöctọ uses postfix notation to describe the composition of functions.

The next example computes `3*7+10`

In [ ]:
run('3 7 * 10 +')

Can you see how the next program computes the same value?

In [ ]:
run('10 3 7 * +')

The language includes blocks that represent quoted programs, i.e., unevaluated programs. Functions like conditionals or loops are able to select the appropriate block and evaluate it when needed

In [ ]:
# if input > 5 then return the execution of 1+2
run('5 > {1 2 +}?',[6])

Herein, function calls are not kept in a call stack, like in typical programming languages. A function call is just the execution of a quoted program. A function call can be replaced by its description inside the main program.

In terms of function composition

$$g = h_1 \circ \ldots \circ h_m$$

$$\text{program} = f_1 \circ f_2 \circ \ldots g \ldots \circ f_n$$

Executing a function call over $g$ is just replacing it with its composition

$$\text{program} = f_1 \circ f_2 \circ \ldots h_1 \circ \ldots \circ h_m \ldots \circ f_n$$

The execution of quoted programs works like a [rewrite system](https://en.wikipedia.org/wiki/Rewriting). This simplifies the management of function calls. And recursivity comes free, being just the replacement of a function call by its own function description.

Let's check some examples with conditionals, loops and function calls to see how they are all special cases of selecting and evaluating quoted programs.

The next is a loop example, the loop pops the current value $n$ and repeats the block for $ï=0,\ldots,n-1$, (ï is the progress variable)

In [ ]:
# prints a triangle of =s (🡗 clears the stack, . outputs the current stack top)
run('{"=" ï 1+* .}F 🡗', [6])  # stack begins in state [6]

In Yöctọ each line is a block, and the engine starts by executing the last line (the block representing the main function),

In [ ]:
# λ0 executes block at line 0, ¹ gets first parameter value, 🡕 duplicates stack top
program = '''
¹ *
1 1 + 🡕 λ0 + 
'''

run(program) # computes (1+1)+(1+1)^2

The next computes the factorial using a cycle,

In [ ]:
# ↑ is the sucessor function
run('1¹{↑*}F',[8])

And this program computes the factorial recursively:

In [ ]:
# n! = (n-1)! * n, ⁇ is the if-then-else command
run('0 = {1} {¹↓λ0¹*}⁇',[8])

## References

+ Wikipedia, [Tacit Programming](https://en.wikipedia.org/wiki/Tacit_programming)

+ Jon Purdy, [Why Concatenative Programming Matters ](https://evincarofautumn.blogspot.com/2012/02/why-concatenative-programming-matters.html?m=1) (2012)

+ Jesse Warden, [Real World Uses of Tacit Programming](https://medium.com/@jesterxl/real-world-uses-of-tacit-programming-part-1-of-2-f2a0c3f9e00c) (2018)

+ [Concatenative.org](https://concatenative.org/wiki/view/Concatenative%20language)
